In [1]:
import cupy as cp
import cusignal
from scipy import signal

import numpy as np

### Generate Sinusodial Signals with N Carriers

**On CPU where**:
* fs = sample rate of signal
* freq = list of carrier frequencies
* N = number of points in signal

In [2]:
def cpu_gen_signal(fs, freq, N):
    T = 1/fs
    sig = 0
    x = np.linspace(0.0, N*(1.0/fs), N)
    for f in freq:
        sig += np.cos(f*2*cp.pi*x)
    return sig

In [3]:
def cpu_gen_ensemble(fs, N, num_sig):
    sig_ensemble = np.zeros((int(num_sig), int(N)))
    for i in range(int(num_sig)):
        # random number of carriers in random locations for each signal
        freq = 1e6 * np.random.randint(1, 10, np.random.randint(1,5))
        sig_ensemble[i,:] = cpu_gen_signal(fs, freq, N)
    return sig_ensemble

**On GPU**

Please note, first run of GPU functions includes setting up memory and 'pre-warming' the GPU. For accurate performance and benchmarking each cell is typically run multiple times.

In [4]:
def gpu_gen_signal(fs, freq, N):
    T = 1/fs
    sig = 0
    x = cp.linspace(0.0, N*(1.0/fs), N)
    for f in freq:
        sig += cp.cos(f*2*cp.pi*x)
    return sig

In [5]:
# Storing num carriers for deep learning prediction -- We're even HURTING ourself here with benchmarks!
def gpu_gen_ensemble(fs, N, num_sig):
    sig_ensemble = cp.zeros((int(num_sig), int(N)))
    num_carriers = cp.zeros(int(num_sig))
    for i in range(int(num_sig)):
        # random number of carriers in random locations for each signal
        num_carrier = int(cp.random.randint(1,5))
        freq = 1e6 * cp.random.randint(1, 10, num_carrier)
        sig_ensemble[i,:] = gpu_gen_signal(fs, freq, N)
        num_carriers[i] = num_carrier
    return sig_ensemble, num_carriers

Generate a bunch of different signals with arbitrary carrier frequencies. Allow user to select number of signals, sample frequency of the ensemble, and number of points in the signal

In [6]:
#10MHz
fs = 10e6

# Overwrite
num_sig = 2000
N = 2**15

# Change sample rate so N=2^16
up = 2
down = 1

In [7]:
cpu_ensemble = cpu_gen_ensemble(fs, N, num_sig)
[gpu_ensemble, num_carriers] = gpu_gen_ensemble(fs, N, num_sig)

### Resample Ensemble - Use Polyphase Resampler to upsample by 2

**On CPU**

In [8]:
%%time
resample_cpu_ensemble = signal.resample_poly(cpu_ensemble, up, down, axis=1, window='flattop')

CPU times: user 3.14 s, sys: 528 ms, total: 3.66 s
Wall time: 3.66 s


**On GPU**

In [9]:
%%time
resample_gpu_ensemble = cusignal.resample_poly(gpu_ensemble, up, down, axis=1, window='flattop')

CPU times: user 279 ms, sys: 16.3 ms, total: 296 ms
Wall time: 294 ms


### Run Periodogram with Flattop Filter over Each Row of Ensemble

**On CPU**

In [11]:
%%time
cf, cPxx_den = signal.periodogram(resample_cpu_ensemble, fs, 'flattop', scaling='spectrum', axis=1)

CPU times: user 3.32 s, sys: 2.2 s, total: 5.52 s
Wall time: 5.52 s


**On GPU**

In [12]:
%%time
gf, gPxx_den = cusignal.periodogram(resample_gpu_ensemble, fs, 'flattop', scaling='spectrum', axis=1)

CPU times: user 199 ms, sys: 68.7 ms, total: 268 ms
Wall time: 272 ms


### Visualize Output

**On CPU**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.semilogy(cf, cPxx_den[0,:])
plt.show()

**On GPU**

In [ ]:
import matplotlib.pyplot as plt
plt.semilogy(cp.asnumpy(gf), cp.asnumpy(gPxx_den[0,:]))
plt.show()

### Move to PyTorch to try to 'predict' number of carriers in signal

In [ ]:
# Uncomment the line below to ensure PyTorch is installed.
# PyTorch is intentionally excluded from our Docker images due to its size.
# Alternatively, the docker image can be run with the following variable:
#     docker run -e EXTRA_CONDA_PACKAGES="-c pytorch pytorch"...

#!conda install -y -c pytorch pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

device = torch.device("cuda:0")

In [ ]:
#90 percent of dataset for training
training_idx_max = int(0.9*gPxx_den.shape[0])

In [ ]:
gPxx_den = gPxx_den.astype(cp.float32)
num_carriers = num_carriers.astype(cp.int64)

# Zero copy memory from cupy to DLPack to Torch
x = torch.as_tensor(gPxx_den[0:training_idx_max,:], device=device)
y = torch.as_tensor(num_carriers[0:training_idx_max], device=device)

# Test
x_t = torch.as_tensor(gPxx_den[training_idx_max:gPxx_den.shape[0],:], device=device)
y_t = torch.as_tensor(num_carriers[training_idx_max:gPxx_den.shape[0]], device=device)

In [ ]:
# Number of possible carriers
output_size = 10

epochs = 75
batch_size = 10
learning_rate = 1e-2

class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
        self.l1 = nn.Linear(x.shape[1], 1500)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(1500, 750)
        self.relu = nn.ReLU()
        self.l5 = nn.Linear(750, output_size)
        
    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l3(x)
        x = self.relu(x)
        x = self.l5(x)
        return F.log_softmax(x, dim=1)
    
net = Network().to(device)

optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.5)

loss_log = []

for e in range(epochs):
    for i in range(0, x.shape[0], batch_size):
        x_mini = x[i:i + batch_size] 
        y_mini = y[i:i + batch_size] 
        
        x_var = Variable(x_mini)
        y_var = Variable(y_mini)
        
        optimizer.zero_grad()
        net_out = net(x_var)
        
        loss = F.nll_loss(net_out, y_var)
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            loss_log.append(loss.data)
        
    print('Epoch: {} - Loss: {:.6f}'.format(e, loss.data))

**Measure Inference Accuracy on Test Set**

In [ ]:
test_loss = 0
correct = 0
for i in range(x_t.shape[0]):
    pred = net(x_t[i,:].expand(1,-1)).argmax()
    correct += pred.eq(y_t[i].view_as(pred)).sum().item()

print('Accuracy: ', 100. * correct / x_t.shape[0])

**Save Model**

In [ ]:
checkpoint = {'net': Network(),
             'state_dict': net.state_dict(),
             'optimizer': optimizer.state_dict()}

torch.save(checkpoint,"E2E_sig_proc.pt")

**Load Model**

In [ ]:
checkpoint = torch.load('E2E_sig_proc.pt')
checkpoint.keys()

**Generate New Signal and Look at Inferencing Power**

In [ ]:
num_carrier = 2
freq = 1e6 * cp.random.randint(1, 10, num_carrier)
sig = gpu_gen_signal(fs, freq, N)
r_sig = cusignal.resample_poly(sig, up, down, window='flattop')
f, Pxx = cusignal.periodogram(r_sig, fs, 'flattop', scaling='spectrum')

x = torch.as_tensor(Pxx.astype(cp.float32), device=device)

pred_num_carrier = net(x.expand(1,-1)).argmax().item()

print(pred_num_carrier)